# AI Mathematical Olympiad - Progress Prize 3
## SC-TIR (Self-Consistency with Tool-Integrated Reasoning) Solution

In [ ]:
# Install required packages (fix version conflicts)
!pip install -q --upgrade numpy==1.26.4
!pip install -q transformers accelerate sympy

In [ ]:
import os
import re
import gc
import time
import traceback
from collections import Counter
from typing import Optional, List
from io import StringIO
from contextlib import redirect_stdout, redirect_stderr

import polars as pl
import torch

IS_SUBMISSION = bool(os.getenv('KAGGLE_IS_COMPETITION_RERUN'))
print(f"Is submission: {IS_SUBMISSION}")

In [ ]:
# Configuration
class Config:
    model_id = "/kaggle/input/math_numina_7b_tir/pytorch/default/1"  # NuminaMath-7B-TIR
    num_samples = 32  # Number of samples for self-consistency
    temperature = 0.7
    max_new_tokens = 2048
    top_p = 0.95
    code_timeout = 10  # seconds

In [ ]:
SYSTEM_PROMPT = """You are a world-class mathematician solving olympiad-level math problems.

IMPORTANT RULES:
1. Think step by step carefully
2. Write Python code to verify calculations when needed
3. Put your code inside ```python and ``` tags
4. Your final answer must be a single non-negative integer
5. Put your FINAL answer inside \\boxed{} at the very end

Example:
Let me solve this step by step...

```python
result = 2 + 2
print(result)
```

The calculation gives 4.
Therefore, the answer is \\boxed{4}
"""

In [ ]:
def extract_code_blocks(text: str) -> List[str]:
    """Extract Python code blocks from text"""
    pattern = r'```python\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches

def execute_code_safely(code: str) -> tuple:
    """Execute code safely and return result"""
    import math
    import cmath
    import fractions
    import itertools
    import functools
    from decimal import Decimal
    
    try:
        import sympy
        import numpy as np
    except:
        sympy = None
        np = None
    
    safe_globals = {
        "__builtins__": {
            "abs": abs, "all": all, "any": any, "bin": bin,
            "bool": bool, "dict": dict, "divmod": divmod,
            "enumerate": enumerate, "filter": filter, "float": float,
            "int": int, "len": len, "list": list, "map": map,
            "max": max, "min": min, "pow": pow, "print": print,
            "range": range, "round": round, "set": set, "sorted": sorted,
            "str": str, "sum": sum, "tuple": tuple, "zip": zip,
            "True": True, "False": False, "None": None,
        },
        "math": math,
        "cmath": cmath,
        "fractions": fractions,
        "Fraction": fractions.Fraction,
        "itertools": itertools,
        "functools": functools,
        "Decimal": Decimal,
    }
    
    if sympy:
        safe_globals["sympy"] = sympy
    if np is not None:
        safe_globals["np"] = np
        safe_globals["numpy"] = np
    
    stdout_capture = StringIO()
    
    try:
        with redirect_stdout(stdout_capture):
            exec(code, safe_globals)
        return True, stdout_capture.getvalue()
    except Exception as e:
        return False, str(e)

In [ ]:
def extract_answer(text: str) -> Optional[int]:
    """Extract final answer from text"""
    # Look for \boxed{...} pattern first
    boxed_pattern = r'\\boxed\{([^}]+)\}'
    matches = re.findall(boxed_pattern, text)
    
    if matches:
        answer_str = matches[-1].strip()
        numbers = re.findall(r'-?\d+', answer_str)
        if numbers:
            try:
                return int(numbers[-1]) % 100000
            except:
                pass
    
    # Look for "answer is X" patterns
    patterns = [
        r'answer\s*(?:is|=|:)\s*(\d+)',
        r'remainder\s*(?:is|=|:)\s*(\d+)',
        r'final\s*answer\s*(?:is|=|:)?\s*(\d+)',
    ]
    
    for pattern in patterns:
        matches = re.findall(pattern, text.lower())
        if matches:
            try:
                return int(matches[-1]) % 100000
            except:
                pass
    
    # Last resort: find last number
    numbers = re.findall(r'\b(\d+)\b', text[-500:])
    if numbers:
        try:
            return int(numbers[-1]) % 100000
        except:
            pass
    
    return None

In [ ]:
# Load model using transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

print("Loading model...")

tokenizer = AutoTokenizer.from_pretrained(
    Config.model_id, 
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    Config.model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Model loaded: {Config.model_id}")
print(f"Device: {model.device}")

In [ ]:
def solve_problem(problem: str) -> int:
    """Solve a math problem using SC-TIR"""
    prompt = f"{SYSTEM_PROMPT}\n\nProblem: {problem}\n\nSolution:"
    
    answers = []
    
    for i in range(Config.num_samples):
        try:
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=Config.max_new_tokens,
                    temperature=Config.temperature,
                    top_p=Config.top_p,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id,
                )
            
            response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
            
            # Execute code blocks
            code_blocks = extract_code_blocks(response)
            for code in code_blocks:
                success, result = execute_code_safely(code)
                if success and result.strip():
                    try:
                        num = int(result.strip().split()[-1])
                        response += f"\nCode output: {num}"
                    except:
                        pass
            
            # Extract answer
            answer = extract_answer(response)
            if answer is not None:
                answers.append(answer)
                
        except Exception as e:
            print(f"  Sample {i+1} error: {e}")
            continue
    
    # Majority voting
    if answers:
        counter = Counter(answers)
        most_common = counter.most_common(1)[0]
        print(f"  Votes: {dict(counter.most_common(5))}")
        return most_common[0]
    
    return 0

In [ ]:
# Load test data
test_path = '/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv'
test_df = pl.read_csv(test_path)
print(f"Loaded {len(test_df)} problems")

In [ ]:
# Solve all problems
results = []

for row in test_df.iter_rows(named=True):
    problem_id = row['id']
    problem = row['problem']
    
    print(f"\n{'='*50}")
    print(f"Solving: {problem_id}")
    print(f"Problem: {problem[:100]}...")
    
    start = time.time()
    answer = solve_problem(problem)
    elapsed = time.time() - start
    
    print(f"Answer: {answer} (took {elapsed:.1f}s)")
    
    results.append({'id': problem_id, 'answer': answer})
    gc.collect()
    torch.cuda.empty_cache()

print(f"\n{'='*50}")
print(f"Completed all {len(results)} problems")

In [ ]:
# Save submission
submission_df = pl.DataFrame(results)
submission_df.write_csv('/kaggle/working/submission.csv')
print("Submission saved!")
print(submission_df)